# Going beyond the basic model: rule weights and percolation
In this notebook, we go beyond the research as described in Scanlon et al (2007). This contains 2 parts:
1. Applying weights to the different (local/global) contributions in the update rule. By doing this, we can evaluate better what their separate effects are, and what remains when one of them is removed completely.
2. Applying percolation analysis methods to the data generated by the CA model of Scanlon et al. By doing this, we investigate whether the truncated power-law clustering possibly lowers the percolation threshold of random two-state 2D grids.

## Setting up

In [ ]:
import sys
from pathlib import Path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import src.CA_model as CA
import src.analysis as an
import src.utils as ut
import numpy as np
import matplotlib.pyplot as plt
import time
from importlib import reload

#### Reload our own modules in case they are updated

In [ ]:
# run to reload CA_model.py and analysis.py and utils.py for updated code
reload(CA)
reload(an)
reload(ut)

## Scanlon rule with different weight on the local and global rule 

In [ ]:
size = 100                          # width and height of the grid
p = 0.5                             # starting fraction of vegetation
update_rule = CA.update_Scanlon_exp # function containing update rule
true_frac=0.3                       # 'natural' (equilibrium) fraction of vegetation
k=3                                 # strength of local interactions
M=20                                 # radius of neighbourhood
N_steps=200                         # number of iterations
skip=100                            # iterations to skip (equilibration period)
seed=0
phis= np.arange(0, 1.1, 0.1)

ut.generate_parallel_phis(size, p, update_rule, true_frac, k, M, N_steps, skip, seed, phis)

In [ ]:

"""
Cluster size distribution using different phi weight, 
Using PARALLELIZE code  
"""

phi_values = np.round(np.arange(0, 1.1, 0.1), 2)
results = {}
final_grids_by_phi = {}

for i, phi in enumerate(phi_values):
    print(f"\nAnalyzing data for phi = {phi}")
    
    # Each phi has a unique seed assigned during generation
    current_seed = seed + i

    # Load the grids for this (phi, seed) pair
    grids = ut.load_data(
        size=size,
        update_rule=update_rule,
        true_frac=true_frac,
        k=k,
        M=M,
        N_steps=N_steps,
        skip=skip,
        seed=current_seed,
        phi=phi,
    )

    #Save final grid for visualization 
    final_grids_by_phi[phi] = grids[-1]

    # Flatten all grids from this file
    combined_grids = list(grids)

    # Compute cluster sizes and fit
    size_list, fit = an.cluster_sizes(combined_grids)
    results[phi] = (size_list, fit)

    # Print parameters
    beta = fit.truncated_power_law.alpha
    s_char = 1 / fit.truncated_power_law.Lambda
    R, p_val = fit.distribution_compare("truncated_power_law", "exponential")

    print(f" beta = {beta:.3f}")
    print(f" s_char = {s_char:.3f}")
    print(f" R = {R:.3f}, p = {p_val:.3f}")

    # Plot with title
    an.plot_cluster_size_distr([size_list], [fit])
    plt.title(f"Cluster size distribution (phi = {phi})")
    plt.show()


#Visualizing the final grids
plt.figure(figsize=(12, 8))

for i, phi in enumerate(phi_values):
    grid = final_grids_by_phi[phi]

    plt.subplot(3, 4, i+1)
    plt.imshow(grid, cmap="Greens")
    plt.title(f"Final grid (phi = {phi})")
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:

"""
Difference rules weight- optimized for faster simulations
Only keeps the last grid instead of storing all the grids for statistics """


phi_values = [0.0, 0.25, 0.5, 0.75, 1] #the different weight of the local rule 
N_evolutions = 4
results = {}
final_grids_by_phi = {} #for visualition

for phi in phi_values: 
    print(f"\nRunning Scalon_exp with phi = {phi}")
    final_grids = []

    for seed in range(N_evolutions): #we run 5 independent CA evolutions for each phi
        start = time.time() #function that returns the time until another point, measure performance 
        grids = CA.evolve_CA(
            size = 90, 
            p = p, 
            update_rule = CA.update_Scanlon_exp, 
            true_frac=true_frac, 
            k=k,
            M = 10, 
            N_steps = 80, 
            skip = 60,
            seed = seed , 
            phi = phi
        )
        final_grids.append(grids[-1])
        end = time.time()
        print(f"Evolution {seed+1}/{N_evolutions} completed in {end-start: .2f} s'")

    final_grids_by_phi[phi] = final_grids 

    #Compute cluster sizes and fit 
    size_list, fit = an.cluster_sizes_safe(final_grids) #size_list gives a list of all cluster sizes, and fit models the distribution 
    
    if len(size_list) ==0: 
        print("No clusters large enough for fitting")
        results[phi]= (size_list,None)
        continue
    
    results[phi] = (size_list, fit)

    #Print parameters
    beta = fit.truncated_power_law.alpha 
    s_char = 1 /fit.truncated_power_law.Lambda
    R, p_val = fit.distribution_compare("truncated_power_law", "exponential") # these are just statistical test to compare the 2 model, if R>0 -> the truncated power law fits better, if p<0.05 then the difference between the fit of the 2 models is statistically significant 

    print(f" beta = {beta:.3f}") 
    print(f" s_char = {s_char:.3f}") 
    print(f" R = {R:.3f}, p = {p_val:.3f}") 
    
    # Plot cluster size distribution for each phi 
    plt.figure(figsize=(6,4))
    an.plot_cluster_size_distr([size_list], [fit])
    plt.title(f"Cluster size distribution (phi = {phi})")
    plt.show()


#Visualizing the final grids
plt.figure(figsize=(12, 8))

for i, phi in enumerate(phi_values):
    grid = final_grids_by_phi[phi][0]  # first seed's final grid

    plt.subplot(2, 3, i+1)
    plt.imshow(grid, cmap="Greens")
    plt.title(f"Final grid (phi = {phi})")
    plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
"""
Plotting of p value as a function of phi
We want to see when does the p value becomes statistically significant (0.05) to say that it's a power law, depending on phi 

"""
phi_list = []
p_values_clean = []

for phi in phi_values:
    size_list, fit = results[phi]
    R, p_val = fit.distribution_compare("truncated_power_law", "exponential")
    phi_list.append(phi)
    p_values_clean.append(p_val if p_val >0 else 1e-16) #we replace them to be able to see them on the graph

plt.figure(figsize=(7,5))
plt.scatter(phi_list, p_values_clean, marker='o')
plt.axhline(0.05, color='red', linestyle='--', label='Significance threshold (0.05)')
plt.xlabel("phi")
plt.ylabel("p-value")
plt.title("Statistical significance of truncated power law vs exponential")
plt.legend()
plt.yscale("log")  # because can be extremely small
plt.show()


In [ ]:
"""
Animation of individual CA evolution with different phis"""
phi_values_to_animate = [0.0, 0.1, 0.2, 0.5, 1.0]

grids_by_phi = {}

for i, phi in enumerate(phi_values_to_animate):
    current_seed = list(phi_values).index(phi)
    grids = ut.load_data(
        size=size,
        update_rule=update_rule,
        true_frac=true_frac,
        k=k,
        M=M,
        N_steps=N_steps,
        skip=skip,
        seed=current_seed,
        phi=phi,
    )
    grids_by_phi[phi] = grids


an.animate_multiple_phis(grids_by_phi, phi_values_to_animate)


## Percolation

In [ ]:
"""
Percolation vs rainfall
We calculate the probablity of percolation for different 
rainfall percentage (true_frac)
"""
rain_values = [0.0, 0.2, 0.4, 0.6, 0.8]
N_evolutions = 20 #20simulations with different seeds 
percs= []

for r in rain_values: 
    total_perc= 0 #count of the percolating grids across all seeds

    for seed in range(N_evolutions): 
        grids = CA.evolve_CA(
            size = 100, 
            p = 0.5, 
            update_rule=CA.update_Scanlon_exp, 
            true_frac=r, 
            k=3, 
            M=10, 
            N_steps=200, 
            skip=100, 
            seed=seed, 
            phi=0.5 ) 
        last_grids = grids[-100:] #taking more grids (final 100) into consideration, 
        
        for g in last_grids: 
            if an.has_vertical_percolation(g) and an.has_horizontal_percolation(g):
                total_perc +=1
    
    #Normaliszation by thte total number of grids looked at 
    percs.append(total_perc/(100* N_evolutions)) #divide by the number of grids we take into consideration, and the number of simulation 

plt.plot(rain_values, percs, marker = 'o')
plt.xlabel("Rainfall (tru_frac)")  
plt.ylabel("Percolation probability") 
plt.title("Percolation threshold") 
plt.show()


In [ ]:
"""
Percolation vs phi (local/global rules) see below percolation trehsold the local rule shows percolation
"""

phi_values = np.round(np.arange(0, 1.1, 0.1), 2)  #reload the data wihth new phi  
rain_values = np.round(np.arange(0.35, 0.66, 0.05), 2) #only around the percolation treshold TRY MORE VALUES 0.59 -0.65
N_evolutions = 5 

plt.figure(figsize=(8,6))

for rain in rain_values: 
    percs = []
    for phi in phi_values: 
        total_perc = 0
        for seed in range(N_evolutions): 
            grids = CA.evolve_CA(
                size = 100, 
                p = p, 
                update_rule=CA.update_Scanlon_exp, 
                true_frac=rain, 
                k=k, 
                M=10, 
                N_steps=100, 
                skip=50, 
                seed=seed, 
                phi=phi) 
            last_grids = grids[-100:] #taking more grids (final 100) into consideration, 
        
            for g in last_grids: 
                if an.has_vertical_percolation(g) or an.has_horizontal_percolation(g):
                    total_perc +=1
                    
        percs.append(total_perc /100* N_evolutions)
    plt.plot(phi_values, percs, marker='o', label=f"Rain = {rain}" )

plt.xlabel("Phi (weight of local rule)")  
plt.ylabel("Percolation probability") 
plt.title(f"Percolation vs Phi (rain fraction = {rain_values})") 
plt.legend()
plt.show()


In [ ]:
"""
Cluster size near the percolation treshold"""

rain_values = [0.4, 0.5, 0.6]  # around treshold region
phi = 0.5
N_evolutions = 5

for rain in rain_values:
    final_grids = []

    for seed in range(N_evolutions):
        grids = CA.evolve_CA(
            size=100,
            p=p,
            update_rule=CA.update_Scanlon_exp,
            true_frac=rain,
            k=k,
            M=10,
            N_steps=100,
            skip=50,
            seed=seed,
            phi=phi
        )
        final_grids.append(grids[-1])

    size_list, fit = an.cluster_sizes(final_grids)
    size_list = [s for s in size_list if s >=5] #only keeps the clusiter sizes >= 5 to fit power law better

    print(f"\nRainfall = {rain}")
    if fit is not None and len(size_list)>0:
        beta = fit.truncated_power_law.alpha
        print(f" alpha = {beta:.3f}")
        an.plot_cluster_size_distr([size_list], [fit])
    else:
        print("No large clusters.")
